In [51]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import random
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.naive_bayes import MultinomialNB

In [46]:
file = open('SMSSpamCollection.txt', 'r',  encoding='utf-8')
A = file.readlines()

Texts = []
Labels = []
for i in range(len(A)):
    if A[i][0] == 'h':
        Texts.append(A[i][4:-1])
        Labels.append(0)
    else: 
        Texts.append(A[i][5:-1])
        Labels.append(1)

In [47]:
v = CountVectorizer()
mx = v.fit_transform(np.array(Texts)).toarray()
random.seed(2)
reg = LogisticRegression()
cross_val_score(reg, mx, Labels, cv = 10, scoring = 'f1').mean()

0.9333485268579415

In [48]:
List = ['FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB',
'FreeMsg: Txt: claim your reward of 3 hours talk time',
'Have you visited the last lecture on physics?','Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$',
'Only 99$']
mx_pred  = v.fit_transform(np.array(Texts+List)).toarray()
reg.fit(mx_pred[:-5], Labels)
reg.predict(mx_pred[-5:])

array([1, 1, 0, 0, 0])

Первые примеры содержат характерные для нашей выборки слова и верно относятся к спаму. Следующие примеры менее характерны.

In [49]:
v1 = CountVectorizer(ngram_range=(2,2))
mx = v1.fit_transform(np.array(Texts)).toarray()
print(cross_val_score(reg, mx, Labels, cv = 10, scoring = 'f1').mean())

v2 = CountVectorizer(ngram_range=(3,3))
mx = v2.fit_transform(np.array(Texts)).toarray()
print(cross_val_score(reg, mx, Labels, cv = 10, scoring = 'f1').mean())

v3 = CountVectorizer(ngram_range=(1,3))
mx = v3.fit_transform(np.array(Texts)).toarray()
print(cross_val_score(reg, mx, Labels, cv = 10, scoring = 'f1').mean())

0.8224220664187133
0.7250161555467377
0.9251382558648837


Обратили внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [52]:
reg = MultinomialNB()

v1 = CountVectorizer(ngram_range=(2,2))
mx = v1.fit_transform(np.array(Texts)).toarray()
print(round(cross_val_score(reg, mx, Labels, cv = 10, scoring = 'f1').mean(),2))

v2 = CountVectorizer(ngram_range=(3,3))
mx = v2.fit_transform(np.array(Texts)).toarray()
print(round(cross_val_score(reg, mx, Labels, cv = 10, scoring = 'f1').mean(),2))

v3 = CountVectorizer(ngram_range=(1,3))
mx = v3.fit_transform(np.array(Texts)).toarray()
print(round(cross_val_score(reg, mx, Labels, cv = 10, scoring = 'f1').mean(),2))

0.65
0.38
0.89


Наивный Байес страдает сильнее от нехватки статистики по биграммам и триграммам.

In [54]:
reg = LogisticRegression()

v = TfidfVectorizer()
mx = v.fit_transform(np.array(Texts)).toarray()
print(round(cross_val_score(reg, mx, Labels, cv = 10, scoring = 'f1').mean(),2))

0.85


Качество понизилось по сравнению с кросс-валидацией CountVectorizer на униграммах